# KarabukWildfire2025 – Analiz Defteri

Tek defter akışı: GEE init, AOI, Sentinel‑2 ön işleme, NDVI/NBR, dNDVI/dNBR, şiddet ve folium haritalar.
Önkoşullar: `pip install -r requirements.txt` ve bir kez `earthengine authenticate`.


## 0. Önkoşullar

Bu defteri çalıştırmadan önce aşağıdaki kurulum adımlarını uygulayın.

- Python 3.10+ önerilir ve izole ortam kullanın (venv).
- Bağımlılıkları `requirements.txt` üzerinden kurun.

Örnek komutlar:

```bash
python -m venv .venv
. .venv/bin/activate  # Windows: .venv\\Scripts\\activate
pip install -r requirements.txt
```

Not: Bağımlılık çatışmalarını önlemek için her projede ayrı bir sanal ortam önerilir.


## 1. Earth Engine Kimlik Doğrulama (bir kez)

Google Earth Engine (GEE) API'yi kullanabilmek için önce kimlik doğrulaması yapın.

- Kullanıcı hesabı ile interaktif doğrulama:

```bash
earthengine authenticate
```

- Alternatif: Servis hesabı ile çalışacaksanız ortam değişkenlerini ayarlayın:

```bash
export EE_SERVICE_ACCOUNT="service-account@project.iam.gserviceaccount.com"
export EE_PRIVATE_KEY_FILE="/path/to/key.json"
export EE_PROJECT="your-project-id"
```

Ardından defterde `ee_init(project=...)` çağrısı yapılır (CLI ve pipeline buna göre çalışır).


In [ ]:
# Importlar
import os
from IPython.display import IFrame, display
import pandas as pd

from src.pipeline import run_pipeline
from src.utils import ee_init


In [ ]:
# Earth Engine proje kimliği (opsiyonel) ve erken başlatma
ee_project = 'karabukwildfire2025'
print('EE_PROJECT =', ee_project or '(not set; you may be prompted to authenticate)')
try:
    ee_init(project=ee_project)
    print('Earth Engine initialized.')
except Exception as e:
    print('Earth Engine init error:', e)


In [ ]:
# Parametreler
pre_start, pre_end = '2025-07-10', '2025-07-25'
post_start, post_end = '2025-07-26', '2025-08-10'
aoi_path = 'src/aoi.geojson'  # yoksa varsayılan Karabük bbox kullanılır
out_dir = 'results'

print('AOI exists? ', os.path.exists(aoi_path))
print('OUT dir   : ', out_dir)
pre_start, pre_end, post_start, post_end


In [ ]:
# Çalıştır
outputs = run_pipeline(
    pre_start=pre_start, pre_end=pre_end,
    post_start=post_start, post_end=post_end,
    aoi_geojson=aoi_path, out_dir=out_dir,
    project=ee_project
)
outputs


In [ ]:
# Haritaları defter içinde görüntüle (opsiyonel)
from IPython.display import HTML

def show_html(path, height=600):
    try:
        display(IFrame(src=path, width='100%', height=height))
    except Exception:
        try:
            with open(path, 'r', encoding='utf-8') as f:
                html = f.read()
            display(HTML(html))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))


In [ ]:
# Özet istatistikler
pd.read_csv(outputs['summary_csv'])


In [ ]:
# Rapor PNG'leri (analiz katmanları + doğal renk)
from src.gee.aoi import get_aoi
from src.gee.preprocess import prepare_composite
from src.gee.indices import with_indices
from src.gee.change import compute_diffs, classify_dnbr
from src.visualize import export_report_pngs, export_truecolor_pngs
from IPython.display import Image, HTML, display
import os, base64

aoi = get_aoi(aoi_path)
pre = with_indices(prepare_composite(aoi, pre_start, pre_end))
post = with_indices(prepare_composite(aoi, post_start, post_end))
diffs = compute_diffs(pre, post)
severity = classify_dnbr(diffs['dNBR'])

pngs = export_report_pngs(pre=pre, post=post, diffs=diffs, severity=severity, aoi=aoi, out_dir=out_dir)
rgbs = export_truecolor_pngs(pre=pre, post=post, aoi=aoi, out_dir=out_dir)

order = [
  ('Pre RGB', rgbs.get('pre_rgb_png')),
  ('Post RGB', rgbs.get('post_rgb_png')),
  ('Pre NDVI', pngs.get('pre_ndvi_png')),
  ('Post NDVI', pngs.get('post_ndvi_png')),
  ('dNBR', pngs.get('dnbr_png')),
  ('Severity', pngs.get('severity_png')),
]

def show_image(path):
    try:
        display(Image(filename=path))
    except Exception:
        try:
            with open(path,'rb') as f:
                data = base64.b64encode(f.read()).decode('ascii')
            display(HTML(f"<img src='data:image/png;base64,{data}' style='max-width:100%;height:auto;'/>"))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))

for title, p in order:
    if not p or not os.path.exists(p):
        continue
    display(HTML(f"<h4 style='margin:8px 0'>{title}</h4>"))
    show_image(p)


In [ ]:
# Toplu çıktı üret ve göster
from IPython.display import HTML, Image, display, IFrame
import os, base64, pandas as pd

# Parametreler (tanımlı değilse varsayılana düş)
try:
    pre_start
    pre_end
    post_start
    post_end
    aoi_path
    out_dir
except NameError:
    pre_start, pre_end = '2025-07-10', '2025-07-25'
    post_start, post_end = '2025-07-26', '2025-08-10'
    aoi_path = 'src/aoi.geojson'
    out_dir = 'results'

from src.pipeline import run_pipeline
from src.gee.aoi import get_aoi
from src.gee.preprocess import prepare_composite
from src.gee.indices import with_indices
from src.gee.change import compute_diffs, classify_dnbr
from src.visualize import export_report_pngs, export_truecolor_pngs

# 1) Pipeline çalıştır (HTML haritalar + summary CSV)
outputs = run_pipeline(
    pre_start=pre_start, pre_end=pre_end,
    post_start=post_start, post_end=post_end,
    aoi_geojson=aoi_path, out_dir=out_dir,
)
display(HTML('<b>Üretilen dosyalar</b>'))
for key, val in list(outputs.items()):
    display(HTML(f"- {key}: <code>{val}</code>"))

# 2) PNG çıktıları üret (rapor ve doğal renk)
aoi = get_aoi(aoi_path)
pre = with_indices(prepare_composite(aoi, pre_start, pre_end))
post = with_indices(prepare_composite(aoi, post_start, post_end))
diffs = compute_diffs(pre, post)
severity = classify_dnbr(diffs['dNBR'])

pngs = export_report_pngs(pre=pre, post=post, diffs=diffs, severity=severity, aoi=aoi, out_dir=out_dir)
rgbs = export_truecolor_pngs(pre=pre, post=post, aoi=aoi, out_dir=out_dir)

order = [
  ('Pre RGB', rgbs.get('pre_rgb_png')),
  ('Post RGB', rgbs.get('post_rgb_png')),
  ('Pre NDVI', pngs.get('pre_ndvi_png')),
  ('Post NDVI', pngs.get('post_ndvi_png')),
  ('dNDVI', pngs.get('dndvi_png')),
  ('dNBR', pngs.get('dnbr_png')),
  ('Severity', pngs.get('severity_png')),
]

def show_image(path: str):
    try:
        display(Image(filename=path))
    except Exception:
        try:
            with open(path,'rb') as f:
                data = base64.b64encode(f.read()).decode('ascii')
            display(HTML(f"<img src='data:image/png;base64,{data}' style='max-width:100%;height:auto;'/>"))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))

display(HTML('<hr/><b>PNG Görseller</b>'))
for title, p in order:
    if not p or not os.path.exists(p):
        continue
    display(HTML(f"<h4 style='margin:8px 0'>{title}</h4>"))
    show_image(p)

# 3) Özet istatistikleri göster
try:
    df = pd.read_csv(outputs['summary_csv'])
    display(HTML('<b>Özet İstatistikler</b>'))
    display(df)
except Exception:
    pass

# 4) İsteğe bağlı: HTML haritalar (IFrame + fallback render)
def show_html(path, height=600):
    try:
        display(IFrame(src=path, width='100%', height=height))
    except Exception:
        try:
            with open(path, 'r', encoding='utf-8') as f:
                html = f.read()
            display(HTML(html))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))

display(HTML('<hr/><b>Haritalar (HTML)</b>'))
for key in ['pre_ndvi_map','post_ndvi_map','pre_nbr_map','post_nbr_map','dnbr_map','severity_map']:
    p = outputs.get(key)
    if p and os.path.exists(p):
        show_html(p, height=600 if 'nbr' in key else 480)
